# ML

Команда: Не творог, а творог

## 10

**Задание**:

Выведите топ-50 самых многочисленных категорий, доступных в классификации этой базы данных.

Выведите топ-50 самых популярных стилей, доступных в классификации этой базы данных.



Импортируем все необходимое:

In [ ]:
import random
import tqdm
import pandas as pd

from lxml import etree

Загрузим и распакуем дамп мастер релизов:

In [ ]:
!wget http://discogs-data-dumps.s3-us-west-2.amazonaws.com/data/2022/discogs_20220601_masters.xml.gz
!gunzip discogs_20220601_masters.xml.gz

--2022-07-10 08:54:24--  http://discogs-data-dumps.s3-us-west-2.amazonaws.com/data/2022/discogs_20220601_masters.xml.gz
Resolving discogs-data-dumps.s3-us-west-2.amazonaws.com (discogs-data-dumps.s3-us-west-2.amazonaws.com)... 52.92.209.178
Connecting to discogs-data-dumps.s3-us-west-2.amazonaws.com (discogs-data-dumps.s3-us-west-2.amazonaws.com)|52.92.209.178|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 462807679 (441M) [application/x-gzip]
Saving to: ‘discogs_20220601_masters.xml.gz’

discogs_20220601_ma 100%[===================>] 441.37M  20.4MB/s    in 23s     

2022-07-10 08:54:47 (19.4 MB/s) - ‘discogs_20220601_masters.xml.gz’ saved [462807679/462807679]



In [ ]:
!ls -lsh

total 2.6G
 72K -rw-r--r-- 1 root root  72K Jul 10 07:52 compilations.csv
2.6G -rw-r--r-- 1 root root 2.6G Jun  8 05:31 discogs_20220601_masters.xml
4.0K drwxr-xr-x 1 root root 4.0K Jul  6 13:22 sample_data
6.8M -rw-r--r-- 1 root root 6.8M Jul 10 07:55 tracks.csv


Данные весят 2.6 Гигабайт! Мои попытки считать их целиком приводили к переполнению оперативной памяти. Придется итерироваться по файлу батчами. Найдем все жанры и стили в файле:

In [ ]:
all_genres = []
all_styles = []

for event, element in tqdm.tqdm(etree.iterparse(
    'discogs_20220601_masters.xml',
    tag='master',
)):
  id = element.get('id')
  for el_c in element.iterchildren():
    if el_c.tag == 'genres':
      for el_c_c in el_c.iterchildren():
        all_genres.append((id, el_c_c.text))
    if el_c.tag == 'styles':
      for el_c_c in el_c.iterchildren():
        all_styles.append((id, el_c_c.text))
  element.clear()

2054691it [01:26, 23764.72it/s]


Создадим pd.Series с жанрами и стилями:

In [ ]:
all_genres_series = pd.DataFrame(all_genres).set_index(0)[1]
all_styles_series = pd.DataFrame(all_styles).set_index(0)[1]

Выведем топ-50 жанров:

In [ ]:
all_genres_series.value_counts().sort_values(ascending=False).head(50)

Rock                      638898
Electronic                580580
Pop                       409947
Folk, World, & Country    249050
Jazz                      164914
Funk / Soul               147053
Classical                 121313
Hip Hop                    99176
Latin                      78698
Reggae                     55912
Stage & Screen             55856
Blues                      44106
Non-Music                  33287
Children's                 18397
Brass & Military            6150
Name: 1, dtype: int64

Выведем топ-50 стилей:

In [ ]:
all_styles_series.value_counts().sort_values(ascending=False).head(50)

House                88638
Pop Rock             87455
Vocal                74872
Punk                 74297
Experimental         68558
Techno               64998
Hardcore             57162
Synth-pop            54403
Indie Rock           54205
Alternative Rock     53197
Soul                 52936
Ambient              52543
Disco                48956
Folk                 48070
Country              46281
Electro              45395
Trance               41993
Ballad               40728
Chanson              40181
Rock & Roll          33940
Downtempo            31750
Psychedelic Rock     30717
Hard Rock            30337
Schlager             30316
Romantic             29155
Black Metal          28369
Funk                 27958
Drum n Bass          26928
Folk Rock            26769
Easy Listening       26614
Heavy Metal          26564
Deep House           26502
Soundtrack           26174
Tech House           25798
Industrial           24045
Rhythm & Blues       23958
Euro House           23731
N

## 20

**Задание**:

Макс Мартин — один из самых знаменитых продюсеров современности. Какие ещё продюсеры могут похвастаться работой с таким количеством исполнителей?

**Примечание**:

В исходных данных (discogs_20220601_masters.xml) информации о продюсерах в явном виде нет (ее можно попытаться достать из описания видео к релизам, но они там плохо структурированы, или из аттрибута join у артистов, но их там мало). 

Для решения этой проблемы, организаторы предложили следующее: "можно взять их (продюсеров) в API, других выгрузках, или если не найдёте — реализуйте поиск самых популярных со-исполнителей (feat), а не продюсеров"

В итоге, удалось достать иформацию о том, является ли артист продюсером или нет из файла discogs_20220601_artists.xml.gz, при помощи простого поиска подстроки 'producer' в описании профиля (так считать немного не корректно, т.к. продюсер мог работать и не в роли продюсера над треком, но имеем, что имеем) :)

Загрузим discogs_20220601_artists.xml.gz:

In [ ]:
!wget http://discogs-data-dumps.s3-us-west-2.amazonaws.com/data/2022/discogs_20220601_artists.xml.gz
!gunzip discogs_20220601_artists.xml.gz

--2022-07-10 08:56:45--  http://discogs-data-dumps.s3-us-west-2.amazonaws.com/data/2022/discogs_20220601_artists.xml.gz
Resolving discogs-data-dumps.s3-us-west-2.amazonaws.com (discogs-data-dumps.s3-us-west-2.amazonaws.com)... 52.218.178.186
Connecting to discogs-data-dumps.s3-us-west-2.amazonaws.com (discogs-data-dumps.s3-us-west-2.amazonaws.com)|52.218.178.186|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 402639192 (384M) [application/x-gzip]
Saving to: ‘discogs_20220601_artists.xml.gz’

discogs_20220601_ar 100%[===================>] 383.99M  21.9MB/s    in 19s     

2022-07-10 08:57:04 (20.2 MB/s) - ‘discogs_20220601_artists.xml.gz’ saved [402639192/402639192]



Составим список продюсеров:

In [ ]:
producers = list()

for event, element in tqdm.tqdm(etree.iterparse(
    'discogs_20220601_artists.xml',
    tag='artist',
)):
  artist_info = dict()

  for el_c in element.iterchildren():
    if el_c.tag == 'id':
      artist_info['id'] = int(el_c.text)
    elif el_c.tag == 'name':
      artist_info['name'] = el_c.text
    elif el_c.tag == 'profile':
      artist_info['is_producer'] = 'producer' in str(el_c.text).lower()

  if artist_info.get('is_producer'):
    producers.append(artist_info)
  element.clear()

8149687it [01:43, 78802.41it/s]


Превратим его в датафрейм:

In [ ]:
producers_df = pd.DataFrame(producers).set_index('id')
producers_df.head()

,name,is_producer
id,,
3,Josh Wink,True
4,Johannes Heil,True
5,Heiko Laux,True
8,Mood II Swing,True
11,DJ Dozia,True


In [ ]:
producers_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153782 entries, 3 to 11333666
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   name         153782 non-null  object
 1   is_producer  153782 non-null  bool  
dtypes: bool(1), object(1)
memory usage: 2.5+ MB


Посмотрим с какими другими артистами работают продюсеры (будем считать уникальных артистов):

In [ ]:
producers_to_artists = {p_id: set() for p_id in producers_df.index}

In [ ]:
for event, element in tqdm.tqdm(etree.iterparse(
    'discogs_20220601_masters.xml',
    tag='master',
)):
  artists = set()

  for el_c in element.iterchildren():
    if el_c.tag == 'artists':
      for el_artist in el_c.iterchildren(tag='artist'):
        for el_artist_c in el_artist.iterchildren():
          if el_artist_c.tag == 'id':
            artists.add(int(el_artist_c.text))
            break
      break

  if len(artists) > 1:
    for a_id in artists:
      if a_id in producers_to_artists:
        producers_to_artists[a_id] = producers_to_artists[a_id] | artists - {a_id}
  element.clear()

2054691it [01:28, 23265.89it/s]


Посчитаем число артистов, с которыми работал каждый продюсер:

In [ ]:
producers_df['co_artists_count'] = producers_df.index.to_series().apply(lambda i: len(producers_to_artists[i]))
producers_df.head()

,name,is_producer,co_artists_count
id,,,
3,Josh Wink,True,9
4,Johannes Heil,True,13
5,Heiko Laux,True,7
8,Mood II Swing,True,15
11,DJ Dozia,True,1


Выведем топ-10 продюсеров по числу актеров, с которыми они работали:

In [ ]:
producers_df.sort_values(by='co_artists_count', ascending=False).head(10)

,name,is_producer,co_artists_count
id,,,
424576,Daniel Barenboim,True,261
282137,National Philharmonic Orchestra,True,186
132084,Snoop Dogg,True,166
274325,Erich Wolfgang Korngold,True,158
9070,Armin van Buuren,True,130
864674,Hans Swarowsky,True,124
833327,Willy Hofmann,True,122
232956,Pitbull,True,118
52833,Frank Sinatra,True,106


А что там у Макса?

In [ ]:
producers_df[producers_df.name == 'Max Martin']

,name,is_producer,co_artists_count
id,,,
61337,Max Martin,True,3


Не с таким уж и большим числом артистов он работал :(

## 30

**Задание**:

Посмотрите на формат Compilation — сборника аудиозаписей. Сборники могут подсказать вам, какие аудиодорожки могут идти друг за другом внутри сборника. По заданной аудиодорожке выводите ранжированный список из 20 других песен других исполнителей, которые чаще всего встречаются вместе с ней в сборниках (не в альбомах одного исполнителя!).

Идея решения была в следующем. Сначала выгрузить список компиляций через API при помощи скрипта:

```python
import requests
import tqdm

QUERY_BASE = 'https://api.discogs.com'
QUERY_KEY = 'YOUR_KEY'
QUERY_SECRET = 'TOUR_SECRET'

query = '?&key=&secret=&'

if __name__ == '__main__':
    master_ids = set()
    for page in tqdm.trange(1, 101):
        result = requests.get(
            url=f'{QUERY_BASE}/database/search',
            params={
                'page': str(page),
                'per_page': str(100),
                'type': 'master',
                'artist': 'Various',
                'country': 'us',
                'format': 'compilation',
                'key': QUERY_KEY,
                'secret': QUERY_SECRET,
            }
        )
        data = result.json()

        for r in data['results']:
            master_ids.add(r['id'])

    with open('compilations.csv', 'w') as f:
        f.write('id\n')
        for id in master_ids:
            f.write(f'{id}\n')
```

Подгрузим данные в коллаб (их можно посмотреть [тут](https://drive.google.com/file/d/16qVNJKUBxhvtRN46BgGYkHyZNb-qCbu8/view?usp=sharing
)):

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=16qVNJKUBxhvtRN46BgGYkHyZNb-qCbu8' -O compilations.csv

--2022-07-10 09:00:37--  https://drive.google.com/uc?export=download&id=16qVNJKUBxhvtRN46BgGYkHyZNb-qCbu8
Resolving drive.google.com (drive.google.com)... 108.177.125.113, 108.177.125.139, 108.177.125.138, ...
Connecting to drive.google.com (drive.google.com)|108.177.125.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-6c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/e7uo8lm4g43ka74lk90dkprhqait4c3b/1657443600000/09237333328454164588/*/16qVNJKUBxhvtRN46BgGYkHyZNb-qCbu8?e=download&uuid=02267a9a-ec67-4b94-8464-d815a4a61c26 [following]
--2022-07-10 09:00:37--  https://doc-04-6c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/e7uo8lm4g43ka74lk90dkprhqait4c3b/1657443600000/09237333328454164588/*/16qVNJKUBxhvtRN46BgGYkHyZNb-qCbu8?e=download&uuid=02267a9a-ec67-4b94-8464-d815a4a61c26
Resolving doc-04-6c-docs.googleusercontent.com (doc-04-6c-docs.googleusercontent.com)... 74.125.203.132, 2

Считаем их:

In [ ]:
compilations_df = pd.read_csv('compilations.csv')
compilations_df.head()

,id
0,1736706
1,1441823
2,1769516
3,1966126
4,1802291


Затем я собирался проитерироваться по ранее выгруженной xml-ке с мастер релизами и вытащить из них список треков, но как оказалось их там нет (истерический смех).

Пришлось их выгружать через API напрямую:


```python
import datetime
import time
from typing import Callable
import tqdm
import requests
import pandas as pd

QUERY_BASE = 'https://api.discogs.com'
QUERY_KEY = 'YOUR_KEY'
QUERY_SECRET = 'TOUR_SECRET'

query = '?&key=&secret=&'


class _DelayWrapper:

    def __init__(self, f: Callable, delay: float) -> None:
        self.f = f
        self.delay = delay
        self.last_call = None

    def __call__(self, *args, **kwargs):
        curr_call = datetime.datetime.now()
        if self.last_call is not None and (curr_call - self.last_call).total_seconds() < self.delay:
            sleep_time = self.delay - \
                (curr_call - self.last_call).total_seconds()
            time.sleep(sleep_time)
            # print(f'Sleep {sleep_time:.5f} s')

        try:
            return self.f(*args, **kwargs)
        finally:
            self.last_call = datetime.datetime.now()


requests.get = _DelayWrapper(f=requests.get, delay=1.01)

if __name__ == '__main__':
    compilations_ids = set(pd.read_csv('compilations.csv').id)
    tracks = list()

    for compilations_id in tqdm.tqdm(compilations_ids):
        try:
            result = requests.get(
                url=f'{QUERY_BASE}/masters/{compilations_id}',
                params={
                    'key': QUERY_KEY,
                    'secret': QUERY_SECRET,
                }
            )
            data = result.json()

            if 'tracklist' not in data:
                print(data)

            for i, track in enumerate(data['tracklist']):
                artist_id, artist_name = None, None
                if track['type_'] != 'track':
                    continue

                if 'artists' not in track:
                    raise Exception(
                        f'{compilations_id} is one artist compilation. Skipping it.')
                for artist in track['artists']:
                    artist_id, artist_name = artist['id'], artist['name']

                tracks.append({
                    'compilation_id': compilations_id,
                    'position': i,
                    'name': track['title'],
                    'artist_id': artist_id,
                    'artist_name': artist_name,
                })
        except Exception as ex:
            print(f'{type(ex)}: {ex}')
            continue

    pd.DataFrame(tracks).to_csv('tracks.csv', index=False)
```

Подгрузим их коллаб (можно скачать еще и [тут](https://drive.google.com/file/d/1KknNykcG7_rosQ3RL9IsQo4lztBZCClJ/view?usp=sharing)):

In [ ]:
!wget 'https://drive.google.com/uc?export=download&id=1KknNykcG7_rosQ3RL9IsQo4lztBZCClJ' -O tracks.csv

--2022-07-10 09:00:38--  https://drive.google.com/uc?export=download&id=1KknNykcG7_rosQ3RL9IsQo4lztBZCClJ
Resolving drive.google.com (drive.google.com)... 108.177.125.100, 108.177.125.138, 108.177.125.113, ...
Connecting to drive.google.com (drive.google.com)|108.177.125.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-6c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/un7o3dg19rqaecknq3elkk6p9bofqmek/1657443600000/09237333328454164588/*/1KknNykcG7_rosQ3RL9IsQo4lztBZCClJ?e=download&uuid=0495ff48-6a0c-4709-8113-dfbb161a9b45 [following]
--2022-07-10 09:00:39--  https://doc-04-6c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/un7o3dg19rqaecknq3elkk6p9bofqmek/1657443600000/09237333328454164588/*/1KknNykcG7_rosQ3RL9IsQo4lztBZCClJ?e=download&uuid=0495ff48-6a0c-4709-8113-dfbb161a9b45
Resolving doc-04-6c-docs.googleusercontent.com (doc-04-6c-docs.googleusercontent.com)... 74.125.203.132, 2

Считаем их:

In [ ]:
tracks_df = pd.read_csv('tracks.csv')
tracks_df.head(5)

,compilation_id,position,name,artist_id,artist_name
0,1736706,1,Ball How Ya Ball,1538984,Mr. Cheefa
1,1736706,2,Pop My P's,108623,Mac Mall
2,1736706,3,Dolla,261580,Haystak
3,1736706,4,Aint No Love,8304643,YT (N.O.H. Mafia)
4,1736706,5,Givin' It Up,376153,Sir Dyno


In [ ]:
tracks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139514 entries, 0 to 139513
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   compilation_id  139514 non-null  int64 
 1   position        139514 non-null  int64 
 2   name            139510 non-null  object
 3   artist_id       139514 non-null  int64 
 4   artist_name     139514 non-null  object
dtypes: int64(3), object(2)
memory usage: 5.3+ MB


Соберем вспомогатьные структуры:

In [ ]:
id_to_artist = dict() # id артиста -> имя артиста
artist_to_id = dict() # имя артиста -> id артиста 
id_to_track = dict() # id трека -> (id артиста, имя трека)
track_to_id = dict() # (id артиста, имя трека) -> id трека
track_id_to_compilation_ids = dict() # id трека -> словарь (id компиляций -> позиция трека)
compilation_id_to_track_ids = dict() # id компиляции -> множество id треков

for r in tqdm.tqdm(tracks_df.itertuples()):
  id_to_artist[r.artist_id] = r.artist_name
  artist_to_id[r.artist_name] = r.artist_id

  track = (r.artist_id, r.name)
  if track not in track_to_id:
    track_id = len(track_to_id)
    track_to_id[track] = track_id
    id_to_track[track_id] = track
  else:
    track_id = track_to_id[track]
  
  track_id_to_compilation_ids[track_id] = {**track_id_to_compilation_ids.get(track_id, dict()), **{r.compilation_id: r.position}}
  compilation_id_to_track_ids[r.compilation_id] = compilation_id_to_track_ids.get(r.compilation_id, set()) | {track_id}

139514it [00:01, 131688.11it/s]


Будем рекомендовать треки других артистов, которые чаще всего встречаются с заданным в компиляции.

Объявим функцию рекомендации:

In [ ]:
def recomend(artist: str, name: str, n: int = 20):
  if artist not in artist_to_id:
    return []
  artist_id = artist_to_id[artist]
  track = (artist_id, name)
  if track not in track_to_id:
    return []
  track_id = track_to_id[track]

  candidates = dict()

  for compilation_id in track_id_to_compilation_ids[track_id]:
    for other_track_id in compilation_id_to_track_ids[compilation_id]:
      if id_to_track[other_track_id][0] != artist_id:
        candidates[other_track_id] = candidates.get(other_track_id, 0) + 1
  
  candidates = sorted(
    map(
      lambda v: (
        (
          v[0],
          (
            id_to_artist[id_to_track[v[0]][0]], 
           id_to_track[v[0]][1]
          ),
        ),
        v[1]
      ),
      candidates.items()
    ), 
    key=lambda v: v[1], 
    reverse=True
  )

  return candidates[:n]

Сделаем рекомендации:

In [ ]:
recomend('Deep Purple', 'Smoke On The Water')

[((4224, ('The Allman Brothers Band', "Ramblin' Man")), 4),
 ((4226, ('Derek & The Dominos', 'Layla')), 3),
 ((4231, ('Alice Cooper', 'Eighteen')), 3),
 ((4241, ('Lynyrd Skynyrd', 'Free Bird')), 3),
 ((4219, ('Grand Funk Railroad', "We're An American Band")), 3),
 ((4225, ('Peter Frampton', 'Show Me The Way')), 2),
 ((770, ('Foreigner', 'Hot Blooded')), 2),
 ((4227, ('The Moody Blues', "I'm Just A Singer (In A Rock And Roll Band)")),
  2),
 ((4229, ('Eric Clapton', 'After Midnight')), 2),
 ((775, ('Golden Earring', 'Radar Love')), 2),
 ((4232, ('Joe Cocker', 'She Came In Through The Bathroom Window')), 2),
 ((4233, ('Stephen Stills', "Love The One You're With")), 2),
 ((4234, ('The Allman Brothers Band', 'Whipping Post')), 2),
 ((4235, ('The Guess Who', 'American Woman')), 2),
 ((4236, ('Canned Heat', 'On The Road Again')), 2),
 ((4237, ('Free', 'All Right Now')), 2),
 ((4238, ('The Yardbirds', 'Heart Full Of Soul')), 2),
 ((4239, ('Joe Walsh', 'Rocky Mountain Way')), 2),
 ((456, ('Cre

In [ ]:
recomend('The O\'Jays', 'Love Train')

[((13662, ('Gladys Knight And The Pips', 'Midnight Train To Georgia')), 4),
 ((3047, ('Average White Band', 'Pick Up The Pieces')), 3),
 ((9059, ('LaBelle', 'Lady Marmalade')), 3),
 ((9314, ('Jim Croce', 'Time In A Bottle')), 3),
 ((22633, ('The Chi-Lites', 'Have You Seen Her')), 3),
 ((656, ('Dobie Gray', 'Drift Away')), 3),
 ((312, ('Commodores', 'Brick House')), 2),
 ((9057, ('The Jackson 5', 'Dancing Machine')), 2),
 ((6021, ('B.T. Express', "Do It ('Til You're Satisfied)")), 2),
 ((465, ('James Taylor (2)', 'Fire And Rain')), 2),
 ((9318, ('Albert Hammond', 'It Never Rains In Southern California')), 2),
 ((4222, ('Rod Stewart', 'Maggie May')), 2),
 ((6019, ('Curtis Mayfield', 'Superfly')), 2),
 ((13654, ('Marvin Gaye', "Let's Get It On")), 2),
 ((13660, ('Four Tops', "Ain't No Woman (Like The One I Got)")), 2),
 ((13665, ('Aretha Franklin', 'Angel')), 2),
 ((22632, ('The Jackson 5', 'A B C')), 2),
 ((22634, ('Billy Paul', 'Me & Mrs. Jones')), 2),
 ((16200, ('Eddie Kendricks', "Kee

Пару песен я проверил руками, вроде, рекомендации неплохие.

## 40

**Задание**:

Придумайте и реализуйте самостоятельно, без использования библиотек, метрику для оценки качества рекомендаций песен. Считайте, что компиляции являются хорошими примерами рекомендаций.

Как мне кажется, метрика качества должна учитывать две вещи:

- **A**. Насколько часто рекомендованные треки встречаются вместе с базовыми (Базовые треки - это треки, к которым строятся рекомендации).
- **B**. Насколько часто рекомендованные треки встречаются друг с другом.

Введем вспомогательные определения:

`P(A)` - вероятность встретить трек A<br>
`P(AB)` - вероятность встретить трек A и трек B вместе<br>
`P(A+B)` - вероятность встретить трек A, или трек B, или оба вместе<br>
`P(B|A)` - вероятность встретить трек B при условии, что встретился трек A (порядок треков не учитывается)<br>

Все эти вероятности можно оценить исходя из компиляций:

`P(A) = |C_A| / |C|`, где C_A - множество компиляций, содержащих трек A; C - множество всех компиляций; |.| - число элементов множества (мощность)<br>
`P(AB) = |C_A & C_B| / |C|`, где X & Y - пересечение множеств<br>
`P(A+B) = |C_A | C_B| / |C|`, где X | Y - объединение множеств<br>
`P(B|A) = P(AB)/P(A) = |C_A & C_B| / |C_A|` (формула условной вероятности)<br>

Учитывая выше сказанное, метрику можно задать следующим образом:

- **A** определим как вероятность того, что при случайном выборе базового трека и рекомендованного трека, рекомендованный трек встретится вместе с базовым. Будем считать следующим образом:<br>
`A = mean(P(r|b) for r in Rec for b in Base) = mean(|C_r & C_b| / |C_b| for r in Rec for b in Base)`
- **B** определим как вероятность того, что при случайном выборе двух рекомендованных треков, они встретятся вместе. Будем считать следующим образом:<br>
`A = mean(P(ab|a+b) for a in Rec for b in Rec) = mean(|C_r & C_b| / |C_A | C_B| for a in Rec for b in Rec)`
- **Score** зададим как среднее арифметическое ранее заданных вероятностей:<br>
`Score = (A + B) / 2`

Примечания:

- Вазовые и рекомендованные треки выбираются из списков базовых и рекомендованных треков соответственно, т.е. уже из сделанной рекомендации.
- Данная метрика не учитывает порядок рекомендаций, но это можно легко исправить, изменив немного формулы.
- В случае, когда рекомендация одна, то вероятность **B** не определена, в этом случае весь **Score** определим как вероятность **A**: `Score = A`

Определим функцию:

In [ ]:
def score(base_ids: list, recomendations_ids: list) -> float:
  score = sum(
    len(set(track_id_to_compilation_ids[br]) & set(track_id_to_compilation_ids[r])) 
    / len(track_id_to_compilation_ids[br])
    for br in base_ids
    for r in recomendations_ids
  ) / len(base_ids) / len(recomendations_ids)

  if len(recomendations_ids) > 1:
    score = (
        score
        + sum(
          len(set(track_id_to_compilation_ids[fr]) & set(track_id_to_compilation_ids[sr])) 
          / len(set(track_id_to_compilation_ids[fr]) | set(track_id_to_compilation_ids[sr]))
          for i, fr in enumerate(recomendations_ids)
          for sr in recomendations_ids[i+1:]
        ) / len(recomendations_ids)
        / (len(recomendations_ids) - 1)
    ) / 2

  return score

Определим вспомогательную функцию для вывода скора и рекомендации:

In [ ]:
def recomend_and_print_score(artist: str, name: str, n: int = 20) -> list:
  recomendations = recomend(artist, name, n)
  base_id = track_to_id[(artist_to_id[artist], name)]
  print('Score', score([base_id], [r[0][0] for r in recomendations]))
  return recomendations

Выведем скор и рекомендации к ранее рассмотренным песням:

In [ ]:
recomend_and_print_score('The O\'Jays', 'Love Train', n=20)

Score 0.0869142881127673


[((13662, ('Gladys Knight And The Pips', 'Midnight Train To Georgia')), 4),
 ((3047, ('Average White Band', 'Pick Up The Pieces')), 3),
 ((9059, ('LaBelle', 'Lady Marmalade')), 3),
 ((9314, ('Jim Croce', 'Time In A Bottle')), 3),
 ((22633, ('The Chi-Lites', 'Have You Seen Her')), 3),
 ((656, ('Dobie Gray', 'Drift Away')), 3),
 ((312, ('Commodores', 'Brick House')), 2),
 ((9057, ('The Jackson 5', 'Dancing Machine')), 2),
 ((6021, ('B.T. Express', "Do It ('Til You're Satisfied)")), 2),
 ((465, ('James Taylor (2)', 'Fire And Rain')), 2),
 ((9318, ('Albert Hammond', 'It Never Rains In Southern California')), 2),
 ((4222, ('Rod Stewart', 'Maggie May')), 2),
 ((6019, ('Curtis Mayfield', 'Superfly')), 2),
 ((13654, ('Marvin Gaye', "Let's Get It On")), 2),
 ((13660, ('Four Tops', "Ain't No Woman (Like The One I Got)")), 2),
 ((13665, ('Aretha Franklin', 'Angel')), 2),
 ((22632, ('The Jackson 5', 'A B C')), 2),
 ((22634, ('Billy Paul', 'Me & Mrs. Jones')), 2),
 ((16200, ('Eddie Kendricks', "Kee

In [ ]:
recomend_and_print_score('Deep Purple', 'Smoke On The Water')

Score 0.2080089146597287


[((4224, ('The Allman Brothers Band', "Ramblin' Man")), 4),
 ((4226, ('Derek & The Dominos', 'Layla')), 3),
 ((4231, ('Alice Cooper', 'Eighteen')), 3),
 ((4241, ('Lynyrd Skynyrd', 'Free Bird')), 3),
 ((4219, ('Grand Funk Railroad', "We're An American Band")), 3),
 ((4225, ('Peter Frampton', 'Show Me The Way')), 2),
 ((770, ('Foreigner', 'Hot Blooded')), 2),
 ((4227, ('The Moody Blues', "I'm Just A Singer (In A Rock And Roll Band)")),
  2),
 ((4229, ('Eric Clapton', 'After Midnight')), 2),
 ((775, ('Golden Earring', 'Radar Love')), 2),
 ((4232, ('Joe Cocker', 'She Came In Through The Bathroom Window')), 2),
 ((4233, ('Stephen Stills', "Love The One You're With")), 2),
 ((4234, ('The Allman Brothers Band', 'Whipping Post')), 2),
 ((4235, ('The Guess Who', 'American Woman')), 2),
 ((4236, ('Canned Heat', 'On The Road Again')), 2),
 ((4237, ('Free', 'All Right Now')), 2),
 ((4238, ('The Yardbirds', 'Heart Full Of Soul')), 2),
 ((4239, ('Joe Walsh', 'Rocky Mountain Way')), 2),
 ((456, ('Cre

Видно, что во втором случае рекомендании оказались лучше.

Сравним со случайными рекомендациями:

In [ ]:
def recomend_random_and_print_score(artist: str, name: str, n: int = 20) -> list:
  recomendations = random.choices(list(id_to_track), k=n)
  base_id = track_to_id[(artist_to_id[artist], name)]
  print('Score', score([base_id], recomendations))
  return recomendations

In [ ]:
recomend_random_and_print_score('The O\'Jays', 'Love Train');
recomend_random_and_print_score('Deep Purple', 'Smoke On The Water');

Score 0.0
Score 0.0035714285714285713


Заметно, что наши рекомендации заметно лучше случайных!

## 50

**Задание**:

Улучшите задание за 30. Основываясь на ссылках на 10 песен подряд, выведите список из 20 других песен, которые с большей вероятностью следуют за ними в компиляциях. Оцените вашу выборку, основываясь на метрике из задания за 40.

Будем рекомендовать треки, которые встречаются после базовых в компиляциях. Будем ранжировать их по тому, как часто они встречаются с базовыми в одной компиляции и как близко они находятся в одной компиляции. 

Определим новую функцию для рекомендаций:

In [ ]:
def recomend_next(base_ids: list, n: int = 20):
  candidates = dict()

  for base_id in base_ids:
    for compilation_id in track_id_to_compilation_ids[base_id]:
      for other_track_id in compilation_id_to_track_ids[compilation_id]:

        base_position = track_id_to_compilation_ids[base_id][compilation_id]
        other_track_position = track_id_to_compilation_ids[other_track_id][compilation_id]

        if base_position < other_track_position:# в задании ничего не сказано про исполнителя, проверять не будем
          # чем раньше встречается трек после базового, тем больше скора даем
          candidates[other_track_id] = candidates.get(other_track_id, 0) + 1 / (other_track_position - base_position)
  
  candidates = sorted(
    map(
      lambda v: (
        (
          v[0],
          (
            id_to_artist[id_to_track[v[0]][0]], 
           id_to_track[v[0]][1]
          ),
        ),
        v[1]
      ),
      candidates.items()
    ), 
    key=lambda v: v[1], 
    reverse=True
  )

  return candidates[:n]

Определим вспомогательную функцию для вывода рекомендаций и скора:

In [ ]:
def recomend_next_and_print_score(base_ids: list, n: int = 20) -> list:
  recomendations = recomend_next(base_ids, n)
  print('Score', score(base_ids, [r[0][0] for r in recomendations]))
  return recomendations

Сделаем рекомендации:

In [ ]:
base = [
  ((13348, ('Sugarloaf', 'Green-Eyed Lady')), 2),
  ((13355, ('Foghat', 'Slow Ride')), 2),
  ((3620, ('Bad Company (3)', "Feel Like Makin' Love")), 2),
  ((3625, ('Grand Funk Railroad', "We're An American Band")), 2),
  ((3628, ('Rod Stewart', 'Maggie May')), 2),
  ((3632, ('The Allman Brothers Band', "Ramblin' Man")), 2),
  ((3633, ('Peter Frampton', 'Show Me The Way')), 2),
  ((3638, ('Golden Earring', 'Radar Love')), 2),
  ((3650, ('Lynyrd Skynyrd', 'Free Bird')), 2),
  ((659, ('The Edgar Winter Group', 'Frankenstein')), 2),
]

In [ ]:
recomend_next_and_print_score([t[0][0] for t in base])

Score 0.3794736842105263


[((3634, ('NASA Space Universe', 'PYOP')), 1.8492063492063493),
 ((3639, ('Bored Straight', 'What Are You Celebrating?')), 1.5244930508088403),
 ((3633, ('Porkeria', 'Somos Punk?')), 1.4019230769230768),
 ((3629, ('Negative Degree', 'Passive-Active')), 1.3611111111111112),
 ((3626, ('Inservibles', 'Viva La Muerte')), 1.1666666666666667),
 ((3635, ('NASA Space Universe', 'Dollah Bill')), 1.1428571428571428),
 ((13356, ('The Project (15)', 'Epilog')), 1.125),
 ((13349, ('Redd Devlin & The Tormentors', 'False Positive')), 1.0),
 ((3621, ('Adjustment To Society', 'Not Yours')), 1.0),
 ((3651, ('Robin S.', 'I Want To Thank You')), 1.0),
 ((34884, ('Frankie Knuckles', 'Too Many Fish')), 1.0),
 ((660, ('Love And Rockets', 'This Heaven')), 1.0),
 ((3640, ('Broken Prayer', 'Growth')), 0.9678571428571429),
 ((3636, ('Good Throb', 'Toner Loner')), 0.8617424242424243),
 ((3630, ('Negative Degree', 'F.O.F.')), 0.8),
 ((3641, ('Thee Nodes', 'Comomodify Me')), 0.756486568986569),
 ((3637, ('Good Thro

Сравним со случайными:

In [ ]:
def recomend_next_random_and_print_score(base_ids: list, n: int = 20) -> list:
  recomendations = random.choices(list(id_to_track), k=n)
  print('Score', score(base_ids, recomendations))
  return recomendations

In [ ]:
recomend_next_random_and_print_score([t[0][0] for t in base]);

Score 0.0


Видно, что наша рекомендация во много раз лучше случайной!